In [2]:
# !pip install pinecone-client

In [47]:
# from embedding import GeminiEmbeddingFunction
import pandas as pd

import os
import io
import re
import time
import glob
from pinecone import Pinecone, ServerlessSpec
from pinecone_embeddings import GeminiEmbeddings
from config.env_variable import EnvVariable
import google.generativeai as genai

API_KEY=EnvVariable.GEMINI_API_KEY.value

genai.configure(api_key=API_KEY)

In [4]:
df = pd.read_parquet("../output_files/text_extracted_circular_df_oct2023toApril2024.parquet")
df.head()

,_id,pageNumber,reference,document,title,timestamp,__v,year,downloaded_pdf_path,downloaded_png_path,text
0,6623ba0dfaf3e17e8b6016dd,1,CCD/FAQ/01/16-03-2024,https://cbn.gov.ng/Out/2024/CCD/FAQs for ferti...,Partnership to Tackle Food Inflation: Fertili...,2024-03-16,0,2024,downloaded_pdfs/FAQs for fertilisers02.pdf,png_files/FAQs for fertilisers02.png,PARTNERSHIP TO TACKLE FOOD INFLATION:\nFERTILI...
1,6623ba0dfaf3e17e8b6016de,1,BSD/DIR/PUB/LAB/017/003,https://cbn.gov.ng/Out/2024/CCD/RE IMPACT OF R...,Re: Impact of Recent Policy Reforms-Prudential...,2024-03-14,0,2024,downloaded_pdfs/RE IMPACT OF RECENT POLICY REF...,png_files/RE IMPACT OF RECENT POLICY REFORMS-P...,Central Bank of Nigeria\n\nBanking Supervision...
2,6623ba0dfaf3e17e8b6016df,1,CCD\SP\GEN\01\14-03-2024,https://cbn.gov.ng/Out/2024/CCD/Governor's Rem...,Handover of Fertilizer to the Ministry of Agri...,2024-03-14,0,2024,downloaded_pdfs/Governor's Remarks At Handover...,png_files/Governor's Remarks At Handover of Fe...,HANDOVER OF FERTILIZER TO THE MINISTRY OF AGRI...
3,6623ba0dfaf3e17e8b6016e0,1,FMD/DIR/PUB/CIR/001/010,https://cbn.gov.ng/Out/2024/FMD/Superceded Cir...,Superseded Circulars/Guidelines (REVIEWED),2024-03-12,0,2024,downloaded_pdfs/Superceded Circulars and Guide...,png_files/Superceded Circulars and Guidelines ...,Central Bank of Nigeria\n\nFinancial Markets D...
4,6623ba0dfaf3e17e8b6016e6,1,OFI/DOA/CON/BDC/006/247,https://cbn.gov.ng/Out/2024/CCD/CBN Press Rele...,"CBN Revokes Operational Licenses of 4,173 BDCs",2024-03-01,0,2024,downloaded_pdfs/CBN Press Release (BDCs Revoca...,png_files/CBN Press Release (BDCs Revocation) ...,```www.cbn.gov.ng\n\nCentral Bank of Nigeria\n...


In [5]:
PINECONE_KEY = EnvVariable.PINECONE_KEY.value
AICX_SEARCH_INDEX = EnvVariable.PINECONE_INDEX.value
recreate_index = True

In [6]:
# PINECONE OBJECT
pc = Pinecone(api_key=PINECONE_KEY)

In [7]:
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

# CREATING AN INDEX
if AICX_SEARCH_INDEX not in pc.list_indexes().names():
	pc.create_index(
		name=AICX_SEARCH_INDEX,
		dimension=768, # gemini embedding dimension
		metric='cosine',
		spec=spec
	)
	time.sleep(1)
	print('INDEX {AICX_SEARCH_INDEX} SUCCESSFULLY CREATED')
else:
	print('INDEX NAME: {AICX_SEARCH_INDEX} ALREADY EXISTS')

index = pc.Index(AICX_SEARCH_INDEX)
index.describe_index_stats()

INDEX NAME: {AICX_SEARCH_INDEX} ALREADY EXISTS


{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [12]:
print(len(embeddings))

20


In [17]:
from tqdm import tqdm
gmd = GeminiEmbeddings()

# embed the texts
embeddings = []
for i, row in tqdm(df.iterrows()):
    try:
        emb = gmd.embed(row["text"])
        embeddings.append(emb)
    except:
        embeddings.append(None)

67it [00:32,  2.06it/s]


In [18]:
df["values"] = embeddings

In [19]:
df.isna().sum()

_id                    0
pageNumber             0
reference              0
document               0
title                  0
timestamp              0
__v                    0
year                   0
downloaded_pdf_path    0
downloaded_png_path    0
text                   0
values                 1
dtype: int64

In [20]:
df = df.dropna()

In [27]:
df.to_csv("../output_files/embedded_texts.csv", index=False)

In [7]:
gmd = GeminiEmbeddings()
prepared_data = gmd.data_prep_pinecone(df)
assert(len(prepared_data)==len(df))

0it [00:00, ?it/s]

In [29]:
def restructure_df(df, metadatas_mapping=None, field_mapping=None):
    if metadatas_mapping is None:
        # default mapping for metadata fields if not provided
        metadatas_mapping = {
            "title":"title",
            "source": "document",
            "date": "timestamp",
            "text": "text"
        }

    if field_mapping is None:
        # default dataframe field mapping if not provided. The values should exist in the provided df
        field_mapping = {
            "values": "values",
            "id": "_id"
        }
    all_docs = []
    for i, row in df.iterrows():
        # Extract metadata, document, and ids data from the dataframe row
        metadata_data = {meta_field: row[df_col] for meta_field, df_col in metadatas_mapping.items()}
        document_data = {meta_field: row[df_col] for meta_field, df_col in field_mapping.items()}
        # document_data = {doc_field: row[df_col] for doc_field, df_col in documents_mapping.items()}
        # ids_data = {ids_field: row[df_col] for ids_field, df_col in ids_mapping.items()}
        
        # Merge all data dictionaries
        document_data["metadata"]=metadata_data

        all_docs.append(document_data)

    return all_docs

In [30]:
document = restructure_df(df)
len(document)

66

In [31]:
index.upsert(
    vectors=document,
    namespace="cbn_circular_v1"
)

{'upserted_count': 66}

__TEST EXAMPLES__

In [33]:
def retrieve_query(query):
    vector = gmd.embed(query)
    results = index.query(
        namespace='cbn_circular_v1',
        vector=vector,
        top_k=2,
        include_metadata=True
        )['matches']

    retrieved_doc = ''
    for i, result in enumerate(results):
        retrieved_doc = retrieved_doc + f'\n\n\n SOURCE DOCUMENT {i + 1} \n\n'
        retrieved_doc = retrieved_doc + ' ' + result['metadata']['text']
    return retrieved_doc

In [45]:
model = genai.GenerativeModel('gemini-pro')

def get_response(query):
  relevant_doc = retrieve_query(query)
  prompt = ("""You are a helpful and informative bot that answers questions relating the policies and circular released by the Central Bank of Nigeria (CBN) betwwen October 2023 till present, April 2024. 
  You are to use the text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=relevant_doc)
  print(prompt)
  answer = model.generate_content(prompt)

  return answer.text


# query = "what is the latest cbn policy on BDC?"
# response = get_response(query)
# response

In [49]:
query = "Why is the Nigerian exchange rate so high"
response = get_response(query)
response

You are a helpful and informative bot that answers questions relating the policies and circular released by the Central Bank of Nigeria (CBN). 
  You are to use the text from the reference passage included below.   Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.   However, you are talking to a non-technical audience, so be sure to break down complicated concepts and   strike a friendly and converstional tone.   If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: 'Why is the Nigerian exchange rate so high'
  PASSAGE: '


 SOURCE DOCUMENT 1 

 ECONOMIC BRIEFING AT THE JOINT SENATE
COMMITTEE ON FINANCE, BANKING, INSURANCE AND
OTHER FINANCIAL INSTITUTIONS AND NATIONAL
PLANNING

A SPEECH PRESENTED BY:

MR. OLAYEMI MICHAEL CARDOSO
GOVERNOR, CENTRAL BANK OF NIGERIA
FRIDAY, FEBRUARY 9TH, 2024

Protocols

- Chairman, Senate Committee on Banking, Insurance, and Other
Financial Institutions: Distinguished Senato

'I am sorry, but this reference text does not address why the Nigerian exchange rate is so high. Therefore, I am unable to answer the question provided.'